<a href="https://colab.research.google.com/github/cluckey-smith/VinBigData_comp_repo/blob/data_processing/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import pandas as pd
import numpy as np
import cv2
from google.colab import drive
drive.mount('/content/drive')
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from keras.layers import Convolution2D
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [2]:
df = pd.read_csv('./train.csv')

In [3]:
train_files = os.listdir('/content/drive/MyDrive/Vin_big_data/train/')

In [162]:
labels = []
x_min_list = []
x_max_list = []
y_min_list = []
y_max_list = []
img_name = []
img = []
bounding_boxes = []
for i,r in df[:1000].iterrows():
  img_name.append(('/content/drive/MyDrive/Vin_big_data/train/') + r['image_id'] + '.png')
  image = load_img(('/content/drive/MyDrive/Vin_big_data/train/') + r['image_id'] + '.png')
  image = img_to_array(image)
  if len(image.shape) > 2:
    image = rgb2gray(image) # Not every image is in grayscale. This is converting them to it
  h,w = image.shape[:2]
  image = image / 255 # Converting our graycale image to values between 0 and 1
  img.append(image)
  labels.append(r['class_name'])
  x_min = r['x_min'] / w
  x_max = r['x_max'] / w
  y_min = r['y_min'] / h
  y_max = r['y_max'] / h
  x_min_list.append(x_min)
  x_max_list.append(x_max)
  y_min_list.append(y_min)
  y_max_list.append(y_max)
  bounding_boxes.append((x_min, y_min, x_max, y_max))
  

# Transforming everything to numpy arrays and performing one-hot encoding for our categorical data

In [163]:
labels = np.array(labels)
bounding_boxes = np.array(bounding_boxes)
img = np.array(img)
img_name = np.array(img_name)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# Filling nan with 0, since some have no diagnoses, and therefore, no bounding box

In [164]:
bounding_boxes = np.nan_to_num(bounding_boxes)

In [165]:
split = train_test_split(img, labels, bounding_boxes, img_name,
	test_size=0.20, random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]

# Needed to add a channel

In [166]:
trainImages = trainImages.reshape(len(trainImages), 512, 512, 1)
testImages = testImages.reshape(len(testImages), 512, 512, 1)

# Super duper basic CNN to just play around and get the feet wet

In [167]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(lb.classes_)))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(trainImages, trainLabels, epochs=10, 
                    validation_data=(testImages, testLabels))

Epoch 1/10
 2/25 [=>............................] - ETA: 4:40 - loss: 5.4210 - accuracy: 0.0625